# Serializing Component

Autogen provides us a way via which we can easily serialize/deserialize component.

In [16]:
import asyncio
from autogen_agentchat.agents import AssistantAgent,UserProxyAgent
from autogen_agentchat.teams import SelectorGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import TextMentionTermination
from dotenv import load_dotenv
import os

# Load API key
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Model client
model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key)

This is useful for debugging, visulaizing and even when we want to share our work with others.

# Termination Condition

In [13]:
from autogen_agentchat.conditions import (MaxMessageTermination,TextMentionTermination)

max_messages_termination = MaxMessageTermination(10)
text_mention_termination = TextMentionTermination("STOP")

final_termination = text_mention_termination & max_messages_termination

final_term_config = final_termination.dump_component()

In [14]:
final_term_config.model_dump_json()

'{"provider":"autogen_agentchat.base.AndTerminationCondition","component_type":"termination","version":1,"component_version":1,"description":null,"label":"AndTerminationCondition","config":{"conditions":[{"provider":"autogen_agentchat.conditions.TextMentionTermination","component_type":"termination","version":1,"component_version":1,"description":"Terminate the conversation if a specific text is mentioned.","label":"TextMentionTermination","config":{"text":"STOP"}},{"provider":"autogen_agentchat.conditions.MaxMessageTermination","component_type":"termination","version":1,"component_version":1,"description":"Terminate the conversation after a maximum number of messages have been exchanged.","label":"MaxMessageTermination","config":{"max_messages":10,"include_agent_event":false}}]}}'

In [15]:
new_final_termination = final_termination.load_component(final_term_config)

# Agents

In [17]:
agent = AssistantAgent(
    name="assistant",
    model_client=model_client,
    system_message="You are a helpful assistant who can solve help to solve tasks.",
)
user_proxy = UserProxyAgent(name="user")

In [18]:
agent.dump_component()
user_proxy.dump_component()

ComponentModel(provider='autogen_agentchat.agents.UserProxyAgent', component_type='agent', version=1, component_version=1, description='An agent that can represent a human user through an input function.', label='UserProxyAgent', config={'name': 'user', 'description': 'A human user'})

In [20]:
import json


with open("user_proxy_dump.json", "w") as f:
    json.dump(user_proxy.dump_component().model_dump(), f, indent=2)

In [21]:
with open("user_proxy_dump.json", "r") as f:
    user_config = json.load(f)

In [22]:
user_agent_new = user_proxy.load_component(user_config)

# Team Serializing

In [24]:
from autogen_agentchat.teams import RoundRobinGroupChat

team = RoundRobinGroupChat(
    participants=[agent, user_agent_new],
    termination_condition=new_final_termination,
)

In [25]:
team_config = team.dump_component()
with open("team_dump.json", "w") as f:
    json.dump(team_config.model_dump(), f, indent=2)

TypeError: Object of type SecretStr is not JSON serializable

### Read about Secret Str in Pydantic Module Python

In [28]:
import json
from pydantic import SecretStr

team_config = team.dump_component()

In [30]:
def serialize_secret(obj):
    if isinstance(obj, SecretStr):
        return obj.get_secret_value()
    raise TypeError(f"Type {type(obj)} not serializable")

with open("team_dump.json", "w") as f:
    json.dump(team_config.model_dump(), f, indent=2,default=serialize_secret)

### Make sure to load components only from Trusted Sources

1. Serialize Components - Debug, Visualize and Share it with others.
2. Termination Condition, Agents, team
3. Team Config into our Autogen Studio
4. We should just load components from a trusted source